In [ ]:
import pyvista as pv
import numpy as np

## Reading Nek files
filename = "./data/v5/po_premix.nek5000"
reader = pv.get_reader(filename)
time_value = 16
reader.set_active_time_value(time_value)

data = reader.read()
times = np.asarray(reader.time_values)
print(times)
## Plotting
## Filtering Data
## Extract Isolevel
iso = data.contour(isosurfaces=3.5, scalars="Temperature")
pl = pv.Plotter()
#pl.add_mesh(data, opacity=0.85)
pl.add_mesh(iso, color='black', line_width=3)
pl.show()
# --- Convert PyVista iso to numpy arrays ---
iso_pts = iso.points  # shape (N_iso, 3)

# --- 2D overlay plot ---
plt.figure(figsize=(5, 8))

# Your SEMDataset-based contour
plt.scatter(
    front["x"], front["y"],
    s=4,
    marker="o",
    edgecolors="none",
    label="SEMDataset contour"
)

# PyVista direct Nek contour
plt.scatter(
    iso_pts[:, 0], iso_pts[:, 1],
    s=1,
    marker="x",
    edgecolors="none",
    label="Nek5000Reader contour"
)

plt.xlabel("x")
plt.ylabel("y")
plt.title(f"Comparison of T = {3.5} isocontours")
plt.ylim(140, 150)
plt.legend()
plt.tight_layout()
plt.show()

# After:
# front = ds.extract_flame_front_unstruct(...)
# iso   = data.contour(...)

# Size of your SEMDataset-based contour (DataFrame)
print(f"SEMDataset isocontour: {len(front)} points")

# Size of Nek5000Reader-based contour (PyVista PolyData)
print(f"Nek5000Reader isocontour: {iso.n_points} points, {iso.n_cells} cells")

# If you want the raw shapes as well:
print("front DataFrame columns:", front.columns.tolist())
print("iso.points.shape:", iso.points.shape)
print("iso.lines.shape:", getattr(iso, 'lines', None).shape if hasattr(iso, 'lines') else None)
print("iso.polys.shape:", getattr(iso, 'polys', None).shape if hasattr(iso, 'polys') else None)

## Many Isocontours
isocon_list = [3.5, 3.8, 4, 4.5, 5]

plt.figure(figsize=(8, 8))

cmap = plt.cm.viridis
colors = cmap(np.linspace(0, 1, len(isocon_list)))

for i, iso in enumerate(isocon_list):
    # Extract the flame front for this isocontour
    isoline = ds.extract_flame_front_unstruct(
        sample_mode="isocontour",
        c_level=iso,
    )

    x = isoline["stretch_rate"].to_numpy()
    y = isoline["disp_speed_density_weighted"].to_numpy()

    plt.scatter(x, y, s=12, color=colors[i], label=f"T = {iso}")

    # Fit a line if possible
    # if len(x) > 2:
    #     coeffs = np.polyfit(x, y, deg=1)
    #     x_line = np.linspace(x.min(), x.max(), 200)
    #     y_line = coeffs[0] * x_line + coeffs[1]
    #     plt.plot(x_line, y_line, color=colors[i], linewidth=2)

plt.xlabel("Stretch rate")
plt.ylabel("Density-weighted displacement speed")
# plt.ylim((-5, 5))
# plt.xlim((-200,200))
plt.title("Sd vs Stretch for Geometric Temperature Isocontours")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

isocon_list = [3.5, 3.8, 4, 4.5, 5]
tol = 0.05
plt.figure(figsize=(8, 8))

cmap = plt.cm.viridis  # choose your preferred colormap
colors = cmap(np.linspace(0, 1, len(isocon_list)))

for i, iso in enumerate(isocon_list):
    # Select iso-line slice
    iso_df = df[(df["T"] > (iso - tol)) & (df["T"] < (iso + tol))]

    x = iso_df["stretch_rate"].to_numpy()
    y = iso_df["disp_speed_density_weighted"].to_numpy()

    # Scatter plot
    plt.scatter(x, y, s=12, color=colors[i], label=f"T ≈ {iso}")

plt.xlabel("Stretch rate")
plt.ylabel("Density-weighted displacement speed")
plt.title("Sd vs Stretch for Temperature Isocontours")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## OLD
front_old = ds.extract_flame_front_old(
    sample_mode="progress",
    c_level=0.51,
    tol=0.35
)
print(front_old)
print("Extarcted flame front")
front_old.to_csv(f"extracted_flame_front_{time_step}.csv", index=False)
plt.figure(figsize=(4, 8))

sc = plt.scatter(
    front_old["x"],
    front_old["y"],
    c=front_old["disp_speed_density_weighted"],
    cmap="viridis",
    linewidths=0.1
)

plt.colorbar(sc, label="Density-weighted displacement speed (Sd)")
plt.xlabel("x")
plt.ylabel("y")
plt.title("Flame Front Colored by Sd")
plt.tight_layout()
plt.show()
import pandas as pd

num_cols = df.select_dtypes(include='number').columns.intersection(
    front.select_dtypes(include='number').columns
)

desc_df = front[num_cols].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99])
desc_front = front_old[num_cols].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99])

# Put them side-by-side
desc_both = pd.concat(
    {'front': desc_df, 'front_old': desc_front},
    axis=1
)

print(desc_both)

from pysemtools.postprocessing.plotting import isosurfaces

# Build mesh dict exactly as the function expects
mesh_dict = {
    "x": ds.x.reshape(-1),
    "y": ds.y.reshape(-1),
    "z": ds.z.reshape(-1),
}

# Choose which field(s) you want isosurfaces for – here: temperature T
fields_dict = {
    "T": ds.t.reshape(-1),  # same shape as mesh coords
}
print(ds.t.shape)
# Define the isosurface levels for each field
isos_dict = {
    "T": np.array([3.5, 3.8, 4.0, 4.5, 5.0]),
}

# Call the pySEMtools helper
plotters = isosurfaces(
    mesh=mesh_dict,
    fields=fields_dict,
    isosurfaces=isos_dict,
    shape=(1, len(isos_dict["T"])),
    colormap="viridis",
    opacity=1.0,
)

# Show each plotter (depending on how you run your code)
for pl in plotters:
    pl.show()